In [48]:
# pip install --upgrade kaggle-environments

In [49]:
# # imports
# !pip install stable-baselines3 pyglet imageio-ffmpeg gymnasium[atari,accept-rom-license] ale-py > /dev/null 2>&1
# !pip install gymnasium==1.0.0
!pip install gymnasium==0.29.1
!pip install ale_py
!pip install stable-baselines3
# !pip install stable-baselines3[extra]
# pip install sb3-contrib

In [50]:
import warnings
warnings.filterwarnings('ignore')

In [51]:
import ale_py

In [52]:
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, VecTransposeImage
from stable_baselines3.common.atari_wrappers import MaxAndSkipEnv
import torch
import numpy as np
import wandb
from wandb.integration.sb3 import WandbCallback

from gymnasium.wrappers import ResizeObservation

In [53]:
print(gym.__version__)

0.29.1


In [54]:
from stable_baselines3.common.monitor import Monitor

In [55]:
import matplotlib.pyplot as plt

In [56]:
import os

In [57]:
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import CheckpointCallback

In [58]:
gym.register_envs(ale_py)

In [59]:
# from sb3_contrib import QRDQN

In [60]:
from datetime import datetime
from stable_baselines3 import A2C
from stable_baselines3.ppo.policies import MlpPolicy
from wandb.integration.sb3 import WandbCallback

In [61]:
from stable_baselines3.common.env_util import make_vec_env

In [62]:
from stable_baselines3 import PPO

In [63]:
from stable_baselines3.common.callbacks import EvalCallback, CallbackList

In [64]:
import collections

In [65]:
from stable_baselines3.common.callbacks import BaseCallback
import torch

In [66]:
from stable_baselines3.common.vec_env import VecEnvWrapper

In [67]:
import cv2

In [68]:
from gymnasium.spaces import Discrete

In [69]:
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback, CallbackList

In [70]:
!nvidia-smi

Wed Dec 11 21:51:35 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   63C    P0             29W /   70W |     159MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [71]:
# configuration file
config = {
    "policy_type": "CnnPolicy",
    "total_timesteps": 1000, # 1000000, 3000000, 20000000
    "Algo": "PPO",
    "env_name": "ALE/DonkeyKong-v5",
    "model_name": "ALE/DonkeyKong-v5",
    "Add": "reduce_action_add_intermediate_reward_position",
    "export_path": "/kaggle/working/exports/",
    "videos_path": "/kaggle/working/videos/",
}



In [72]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANDB_API_KEY ")

In [ ]:
# Wandb setup
wandb.login(key=secret_value_0)
run = wandb.init(
    project="Prova",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    save_code=True,  # optional
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
class MyVecTransposeImage(VecEnvWrapper):
    def __init__(self, venv, skip=False):
        super().__init__(venv)
        self.skip = skip

        # Get original shape: e.g., (84, 84, 4)
        old_shape = self.observation_space.shape
        # Transpose shape to (C, H, W)
        new_shape = (old_shape[2], old_shape[0], old_shape[1])  # (4, 84, 84)

        # Use the original low/high if they are uniform; if not, use min/max appropriately
        low_val = self.observation_space.low.min()
        high_val = self.observation_space.high.max()

        self.observation_space = gym.spaces.Box(
            low=low_val,
            high=high_val,
            shape=new_shape,
            dtype=self.observation_space.dtype
        )

    def reset(self):
        obs = self.venv.reset()
        return self.transpose_observations(obs)

    def step_async(self, actions):
        self.venv.step_async(actions)

    def step_wait(self):
        obs, rewards, dones, infos = self.venv.step_wait()
        return self.transpose_observations(obs), rewards, dones, infos

    def transpose_observations(self, obs):
        if self.skip:
            return obs
        if isinstance(obs, dict):
            for key, val in obs.items():
                obs[key] = self._transpose(val)
            return obs
        else:
            return self._transpose(obs)

    def _transpose(self, obs):
        # obs shape is (n_envs, H, W, C) -> transpose to (n_envs, C, H, W)
        return obs.transpose(0, 3, 1, 2)


In [ ]:
def get_agent_level_position(image):
    if image is None:
        raise ValueError("Image not loaded. Check the path and file.")
    
    # remove 0 to 25 pixels from the top
    image = image[32:, :]
    # plt.imshow(image, cmap='gray')

    image[149:160, 36:44] = 0
    # display image with black
    # plt.imshow(image, cmap='gray')

    # Lines detection
    # copy image
    gray_image = image.copy()

    # print("Image shape:", gray_image.shape)

    # Perform edge detection
    edges = cv2.Canny(gray_image, threshold1=30, threshold2=100)

    # Debug: Show edges
    # plt.figure(figsize=(6, 4))
    # plt.imshow(edges, cmap='gray')
    # plt.title("Edges")
    # plt.axis("off")
    # plt.show()

    # Detect horizontal lines using Hough Transform
    lines = cv2.HoughLinesP(
        edges, 
        rho=1, 
        theta=np.pi / 180, 
        threshold=30, 
        minLineLength=10, 
        maxLineGap=20
    )

    # Draw detected lines on a debug image
    debug_line_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    horizontal_lines = []

    if lines is not None:
        # print(f"Total lines detected (before filtering): {len(lines)}")
        for line in lines:
            x1, y1, x2, y2 = line[0]
            # Check for horizontal lines with a more lenient threshold
            vertical_diff = abs(y2 - y1)
            horizontal_diff = abs(x2 - x1)
            
            if vertical_diff < horizontal_diff * 0.1:  # Allow slight vertical tilt
                horizontal_lines.append((x1, y1, x2, y2))
                # cv2.line(debug_line_image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue for horizontal lines
    # else:
    #     print("No lines detected.")

    # Debug: Show all detected horizontal lines
    # plt.figure(figsize=(10, 6))
    # plt.imshow(cv2.cvtColor(debug_line_image, cv2.COLOR_BGR2RGB))
    # plt.title("Detected Horizontal Lines (Blue)")
    # plt.axis("off")
    # plt.show()

    # Print detected horizontal lines for debugging
    # print(f"Total horizontal lines detected (after filtering): {len(horizontal_lines)}")
    # for idx, (x1, y1, x2, y2) in enumerate(horizontal_lines):
    #     print(f"Line {idx + 1}: ({x1}, {y1}) to ({x2}, {y2})")

    # detect the agent and it position
    # Perform binary thresholding to highlight the agent and objects
    _, binary = cv2.threshold(gray_image, 50, 255, cv2.THRESH_BINARY)

    # Apply morphological operations to clean up noise
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    binary_cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    # Debug: Show the binary image after cleaning
    # plt.figure(figsize=(6, 4))
    # plt.imshow(binary_cleaned, cmap='gray')
    # plt.title("Binary Image (Cleaned)")
    # plt.axis("off")
    # plt.show()

    # Detect contours in the cleaned binary image
    contours, _ = cv2.findContours(binary_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize variables for the agent's position
    agent_detection_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    agent_position = None

    # Filter contours to find the agent
    # print(f"Total contours detected: {len(contours)}")

    for contour in contours:
        # Get bounding box of the contour
        x, y, w, h = cv2.boundingRect(contour)
        
        # Filter based on size: Assuming agent is small
        if 5 <= w <= 30 and 5 <= h <= 30:  # Adjust range based on resolution
            # Further filter based on aspect ratio to avoid line-like objects
            aspect_ratio = max(w / h, h / w)
            if aspect_ratio < 2.0:  # Allow only nearly square contours
                agent_position = (x + w // 2, y + h // 2)  # Center of the bounding box
                
                # Draw the bounding box and center point for the agent
                # cv2.rectangle(agent_detection_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                # cv2.circle(agent_detection_image, agent_position, radius=5, color=(0, 0, 255), thickness=-1)
                # print(f"Agent detected at: {agent_position}, size: {w}x{h}")
                break  # Assuming only one agent in the frame

    # if agent_position is None:
    #     print("No agent detected. Try adjusting size and aspect ratio filters.")

    # Display the agent detection result
    # plt.figure(figsize=(10, 6))
    # plt.imshow(cv2.cvtColor(agent_detection_image, cv2.COLOR_BGR2RGB))
    # plt.title("Agent Detection")
    # plt.axis("off")
    # plt.show()

    
    # Detect level
    # Sort lines by their average y-value, descending (bottom to top)
    lines_sorted = sorted(horizontal_lines, key=lambda line: (line[1] + line[3]) / 2, reverse=True)

    def cluster_lines(lines, desired_clusters=7, proximity_threshold=10):
        clusters = []
        current_cluster = [lines[0]]
        for line in lines[1:]:
            line_y = (line[1] + line[3]) // 2
            current_cluster_y = sum((l[1]+l[3])//2 for l in current_cluster) / len(current_cluster)
            # If the difference is small, add to current cluster, else start a new one
            if abs(line_y - current_cluster_y) < proximity_threshold:
                current_cluster.append(line)
            else:
                clusters.append(current_cluster)
                current_cluster = [line]
        clusters.append(current_cluster)
        
        # If we get more or fewer than desired_clusters, adjust proximity_threshold and try again
        # For simplicity, if we don't get exactly 7, we might raise an error or adjust threshold.
        # But here we assume we pick a threshold that works.
        # if len(clusters) != desired_clusters:
        #     # Adjust proximity_threshold manually or implement a loop to find a suitable threshold.
        #     print(f"Expected {desired_clusters} clusters, got {len(clusters)}. Adjust proximity_threshold.")
        return clusters

    proximity_threshold = 10  # Adjust as needed
    clusters = cluster_lines(lines_sorted, desired_clusters=7, proximity_threshold=proximity_threshold)

    # Compute representative y-value for each cluster (average)
    boundary_y_values = []
    for cluster in clusters:
        avg_y = sum((l[1] + l[3]) // 2 for l in cluster) / len(cluster)
        boundary_y_values.append(avg_y)

    # Sort boundaries again in descending order (bottom = largest y, top = smallest y)
    boundary_y_values.sort(reverse=True)

    # Now we have 7 boundaries for 6 levels:
    # Level 1: between boundary_y_values[0] and boundary_y_values[1]
    # Level 2: between boundary_y_values[1] and boundary_y_values[2]
    # ...
    # Level 6: between boundary_y_values[5] and boundary_y_values[6]

    agent_level = None
    if agent_position:
        agent_y = agent_position[1]
        # Find which level agent_y falls into
        for i in range(6):
            if boundary_y_values[i] >= agent_y > boundary_y_values[i+1]:
                agent_level = i + 1
                break

    # Print only the agent's level
    # if agent_level is not None:
    #     print(agent_level)
    # else:
    #     print("Could not determine agent's level.")
    # return_not_detected_agent = 0
    # if agent_level is None:
    #     # print("Could not determine agent's level.")
    #     return_not_detected_agent = 1

    # Draw minimal annotation: just draw the agent and print its level
    final_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    # if agent_position:
    #     cv2.circle(final_image, agent_position, radius=5, color=(0, 0, 255), thickness=-1)
    #     if agent_level is not None:
    #         cv2.putText(
    #             final_image, 
    #             f"Level: {agent_level}", 
    #             (agent_position[0] + 10, agent_position[1] - 10), 
    #             cv2.FONT_HERSHEY_SIMPLEX, 
    #             0.5, 
    #             (0, 255, 0), 
    #             1
    #         )

    # plt.figure(figsize=(10, 6))
    # plt.imshow(cv2.cvtColor(final_image, cv2.COLOR_BGR2RGB))
    # plt.axis("off")
    # plt.show()

    return agent_level, agent_position# agent position is (x, y)

In [ ]:
class IntermediateRewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super(IntermediateRewardWrapper, self).__init__(env)
        self.ladder_postion = [110,82,90,70,110,78]
        self.last_level = 1
        self.previous_additional_reward = 0.0
        self.last_y_position = 0

    def step(self, action):

        obs, reward, terminated, truncated, info = self.env.step(action)
        # print(f"from intermediate reward {obs.shape}")
        # print(f"from intermediate reward {reward}")

        # done = terminated or truncated

        additional_reward = 0.0
        # get agent level and position
        agent_level, agent_position = get_agent_level_position(obs)
        


        if agent_level is not None and 1 <= agent_level <= len(self.ladder_postion):
            agent_level_reward = (7 - agent_level)* (-0.01)
            additional_reward += agent_level_reward
            
            if agent_level > self.last_level:
                additional_reward += 500.0
                self.last_level = agent_level

            diff = 0
            if agent_position is not None and len(agent_position) > 0:
                # get absolute difference between agent position and ladder position
                diff = abs(self.ladder_postion[agent_level - 1] - agent_position[0])
                agent_position_reward = diff * (-0.1)
                additional_reward += agent_position_reward 
            
                if action == 2 and diff <= 1 and agent_position[1] > self.last_y_position:
                    additional_reward += 25.0
                self.last_y_position = agent_position[1]
            


        else:
            # If agent_level or agent_position not found, use the previous reward
            additional_reward = self.previous_additional_reward


        # Round the additional reward to 2 decimal places
        additional_reward = round(additional_reward, 2)

        # Update the previous reward
        self.previous_additional_reward = additional_reward

        # Add the additional reward to the original reward
        reward += additional_reward

        # print(f"obs: {obs.shape} , Agent Level: {agent_level}, Agent Position: {agent_position}, Additional Reward: {additional_reward}, Total Reward: {reward}")
        
        return obs, reward, terminated, truncated, info


In [ ]:
class ActionFilterWrapper(gym.ActionWrapper):
    def __init__(self, env, allowed_actions):
        super().__init__(env)
        self.allowed_actions = allowed_actions
        # The new action space matches the number of allowed actions
        self.action_space = Discrete(len(self.allowed_actions))

    def action(self, act):
        # Map the reduced action space index to the original action
        return self.allowed_actions[act]

In [ ]:
class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        # The original shape remains (84,84,1), but the dtype and range change
        self.observation_space = gym.spaces.Box(
            low=0.0,
            high=1.0,
            shape=self.observation_space.shape,
            dtype=np.float32
        )
        
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0


# class ScaledFloatFrame(gym.ObservationWrapper):
#     def observation(self, obs):
#         return np.array(obs).astype(np.float32) / 255.0



class FireResetEnv(gym.Wrapper):
    def __init__(self, env=None):
        super().__init__(env)
        # Check that 'FIRE' is a valid action in the environment
        assert 'FIRE' in env.unwrapped.get_action_meanings(), "Environment does not support 'FIRE' action"
        assert len(env.unwrapped.get_action_meanings()) >= 3, "Action space too small for expected actions"

    def step(self, action):
        return self.env.step(action)

    def reset(self, **kwargs):
        # Reset the environment
        obs, info = self.env.reset(**kwargs)

        # Perform the FIRE action
        obs, _, terminated, truncated, _ = self.env.step(1)
        if terminated or truncated:  # If game ends after FIRE, reset again
            obs, info = self.env.reset(**kwargs)

        return obs, info
        
# Custom wrapper to add channel dimension
class AddChannelDimension(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape
        # Update the observation space to include a channel dimension
        self.observation_space = gym.spaces.Box(
            low=0,
            high=255,
            shape=(obs_shape[0], obs_shape[1], 1),
            dtype=np.uint8,
        )

    def observation(self, observation):
        # Add a channel dimension
        return np.expand_dims(observation, axis=-1)

# class MaxAndSkipEnv(gym.Wrapper):
#     def __init__(self, env=None, skip=4):
#         super(MaxAndSkipEnv, self).__init__(env)
#         self._obs_buffer = collections.deque(maxlen=2)
#         self._skip = skip

#     def step(self, action):
#         total_reward = 0.0
#         done = None
#         for _ in range(self._skip):
#             obs, reward, terminated,truncated, info = self.env.step(action)
#             done = terminated or truncated
#             self._obs_buffer.append(obs)
#             total_reward += reward
#             if done:
#                 break
#         max_frame = np.max(np.stack(self._obs_buffer), axis=0)
#         return max_frame, total_reward, terminated, truncated, info

    # def reset(self, *, seed=None, options=None):
    #     self._obs_buffer.clear()
    #     obs, info = self.env.reset(seed=seed, options=options)
    #     self._obs_buffer.append(obs)
    #     return obs, info


def make_env(env_name, allowed_actions, obs_type="grayscale", render_mode=None,):
    def _init():
        env = gym.make(env_name, obs_type="grayscale", render_mode=render_mode)
        print("Standard Env.        : {}".format(env.observation_space.shape))
        env = FireResetEnv(env)
        print("FireResetEnv          : {}".format(env.observation_space.shape))
        # Wrap the environment with the custom ActionFilterWrapper
        env = ActionFilterWrapper(env, allowed_actions)
        print("ActionFilterWrapper   : {}".format(env.observation_space.shape))
        # Wrap the environment to add intermediate rewards
        env = IntermediateRewardWrapper(env)
        print("IntermediateReward    : {}".format(env.observation_space.shape))
        # env = MaxAndSkipEnv(env)
        # print("MaxAndSkipEnv        : {}".format(env.observation_space.shape))
        env = ResizeObservation(env, (84, 84))
        print("ResizeObservation    : {}".format(env.observation_space.shape))
        env = AddChannelDimension(env)  # Add channel dimension here
        print("AddChannelDimension  : {}".format(env.observation_space.shape))
        
        env = ScaledFloatFrame(env)
        print("ScaledFloatFrame     : {}".format(env.observation_space.shape))
        
        # env = Monitor(env, allow_early_resets=False) # from stable baselines
        # print("Monitor               : {}".format(env.observation_space.shape))

        return env
    return _init


# select relevant actions
allowed_actions = [0, 1, 2, 3, 4, 5, 11, 12]
# env = DummyVecEnv([make_env(config["env_name"])])
# use make_vec_env with n_envs=16
env = make_vec_env(env_id=make_env(config["env_name"], allowed_actions= allowed_actions), n_envs=8)

# stack 4 frames
env = VecFrameStack(env, n_stack=4)
print("Post VecFrameStack Shape: {}".format(env.observation_space.shape))

# convert back to PyTorch format (channel-first)
# env = VecTransposeImage(env)
env = MyVecTransposeImage(env)
print("VecTransposeImage Shape: {}".format(env.observation_space.shape))

print("Final Observation Space: {}".format(env.observation_space.shape))

# Start Testing

In [ ]:
# class ScaledFloatFrame(gym.ObservationWrapper):
#     def __init__(self, env):
#         super().__init__(env)
#         # The original shape remains (84,84,1), but the dtype and range change
#         self.observation_space = gym.spaces.Box(
#             low=0.0,
#             high=1.0,
#             shape=self.observation_space.shape,
#             dtype=np.float32
#         )
        
#     def observation(self, obs):
#         return np.array(obs).astype(np.float32) / 255.0


# # class ScaledFloatFrame(gym.ObservationWrapper):
# #     def observation(self, obs):
# #         return np.array(obs).astype(np.float32) / 255.0



# class FireResetEnv(gym.Wrapper):
#     def __init__(self, env=None):
#         super().__init__(env)
#         # Check that 'FIRE' is a valid action in the environment
#         assert 'FIRE' in env.unwrapped.get_action_meanings(), "Environment does not support 'FIRE' action"
#         assert len(env.unwrapped.get_action_meanings()) >= 3, "Action space too small for expected actions"

#     def step(self, action):
#         return self.env.step(action)

#     def reset(self, **kwargs):
#         # Reset the environment
#         obs, info = self.env.reset(**kwargs)

#         # Perform the FIRE action
#         obs, _, terminated, truncated, _ = self.env.step(1)
#         if terminated or truncated:  # If game ends after FIRE, reset again
#             obs, info = self.env.reset(**kwargs)

#         return obs, info
        
# # Custom wrapper to add channel dimension
# class AddChannelDimension(gym.ObservationWrapper):
#     def __init__(self, env):
#         super().__init__(env)
#         obs_shape = self.observation_space.shape
#         # Update the observation space to include a channel dimension
#         self.observation_space = gym.spaces.Box(
#             low=0,
#             high=255,
#             shape=(obs_shape[0], obs_shape[1], 1),
#             dtype=np.uint8,
#         )

#     def observation(self, observation):
#         # Add a channel dimension
#         return np.expand_dims(observation, axis=-1)

# # class MaxAndSkipEnv(gym.Wrapper):
# #     def __init__(self, env=None, skip=4):
# #         super(MaxAndSkipEnv, self).__init__(env)
# #         self._obs_buffer = collections.deque(maxlen=2)
# #         self._skip = skip

# #     def step(self, action):
# #         total_reward = 0.0
# #         done = None
# #         for _ in range(self._skip):
# #             obs, reward, terminated,truncated, info = self.env.step(action)
# #             done = terminated or truncated
# #             self._obs_buffer.append(obs)
# #             total_reward += reward
# #             if done:
# #                 break
# #         max_frame = np.max(np.stack(self._obs_buffer), axis=0)
# #         return max_frame, total_reward, terminated, truncated, info

#     # def reset(self, *, seed=None, options=None):
#     #     self._obs_buffer.clear()
#     #     obs, info = self.env.reset(seed=seed, options=options)
#     #     self._obs_buffer.append(obs)
#     #     return obs, info


# def make_env(env_name, obs_type="grayscale", render_mode=None,):
#     def _init():
#         env = gym.make(env_name, obs_type="grayscale", render_mode=render_mode)
#         print("Standard Env.        : {}".format(env.observation_space.shape))
#         env = FireResetEnv(env)
#         print("FireResetEnv          : {}".format(env.observation_space.shape))
#         # env = MaxAndSkipEnv(env)
#         # print("MaxAndSkipEnv        : {}".format(env.observation_space.shape))
#         env = ResizeObservation(env, (84, 84))
#         print("ResizeObservation    : {}".format(env.observation_space.shape))
#         env = AddChannelDimension(env)  # Add channel dimension here
#         print("AddChannelDimension  : {}".format(env.observation_space.shape))
#         # env = GrayscaleObservation(env, keep_dim=True)
#         # print("GrayscaleObservation : {}".format(env.observation_space.shape))
        
#         env = ScaledFloatFrame(env)
#         print("ScaledFloatFrame     : {}".format(env.observation_space.shape))
        
#         # env = Monitor(env, allow_early_resets=False) # from stable baselines
#         # print("Monitor               : {}".format(env.observation_space.shape))

#         return env
#     return _init



# # env = DummyVecEnv([make_env(config["env_name"])])
# # use make_vec_env with n_envs=16
# env = make_vec_env(env_id=make_env(config["env_name"]), n_envs=1) # ALE/MarioBros-v5

# # stack 4 frames
# env = VecFrameStack(env, n_stack=4)
# print("Post VecFrameStack Shape: {}".format(env.observation_space.shape))

# # convert back to PyTorch format (channel-first)
# # env = VecTransposeImage(env)
# env = MyVecTransposeImage(env)
# print("VecTransposeImage Shape: {}".format(env.observation_space.shape))

# print("Final Observation Space: {}".format(env.observation_space.shape))

In [ ]:
# obs = env.reset()
# print("Initial observation shape:", obs.shape)
# done = False
# total_reward = 0.0

# for i in range(10000):
#     # Take a random action
#     action = env.action_space.sample()
#     obs, reward, done, info = env.step([action])
#     total_reward += reward

#     # if i % 50 == 0:
#     #     print(f"Step: {i}, Reward: {reward}, Done: {done}, Obs mean: {obs.mean()}")
#     if reward>0:
#         print(f"Step: {i}, Reward: {reward}, Done: {done}, Obs mean: {obs.mean()}")

#     if done:
#         print("Done")
#         obs = env.reset()
#         total_reward = 0.0

In [ ]:
# class ScaledFloatFrame(gym.ObservationWrapper):
#     def __init__(self, env):
#         super().__init__(env)
#         # The original shape remains (84,84,1), but the dtype and range change
#         self.observation_space = gym.spaces.Box(
#             low=0.0,
#             high=1.0,
#             shape=self.observation_space.shape,
#             dtype=np.float32
#         )
        
#     def observation(self, obs):
#         return np.array(obs).astype(np.float32) / 255.0

In [ ]:
# def make_env(env_name, obs_type="grayscale", render_mode=None,):
#     def _init():
#         env = gym.make(env_name, obs_type="grayscale", render_mode=render_mode)
#         print("Standard Env.        : {}".format(env.observation_space.shape))
#         env = FireResetEnv(env)
#         print("FireResetEnv          : {}".format(env.observation_space.shape))
#         env = ResizeObservation(env, (84, 84))
#         print("ResizeObservation    : {}".format(env.observation_space.shape))
#         env = ResizeObservation(env, (84, 84))
#         print("ResizeObservation    : {}".format(env.observation_space.shape))
#         env = AddChannelDimension(env)  # Add channel dimension here
#         print("AddChannelDimension  : {}".format(env.observation_space.shape))
#         env = ScaledFloatFrame(env)
#         print("ScaledFloatFrame     : {}".format(env.observation_space.shape))

#         return env
#     return _init

In [ ]:
# # env = gym.make("ALE/Pong-v5", obs_type="grayscale")
# # print("Standard Env.        : {}".format(env.observation_space.shape))
# # env = FireResetEnv(env)
# # print("FireResetEnv          : {}".format(env.observation_space.shape))
# # env = ResizeObservation(env, (84, 84))
# # print("ResizeObservation    : {}".format(env.observation_space.shape))
# # env = AddChannelDimension(env)
# # print("AddChannelDimension   : {}".format(env.observation_space.shape))
# # env = ScaledFloatFrame(env)
# # print("ScaledFloatFrame     : {}".format(env.observation_space.shape))
# # env = Monitor(env, allow_early_resets=False) # from stable baselines
# # print("Monitor               : {}".format(env.observation_space.shape))
# env = make_vec_env(env_id=make_env(config["env_name"]), n_envs=1)
# obs = env.reset()
# print(obs.shape)
# print(obs)
# print("Initial Obs Stats -> min:", obs.min(), "max:", obs.max(), "mean:", obs.mean())

# End Testing

In [ ]:
print("Check")
print("Post VecFrameStack Shape: {}".format(env.observation_space.shape))
print("Final Observation Space: {}".format(env.observation_space.shape))

In [ ]:
# def make_env_evual(env_name, obs_type="grayscale", render_mode=None,):
#     def _init():
#         env = gym.make(env_name, obs_type="grayscale", render_mode=render_mode)
#         print("Standard Env.        : {}".format(env.observation_space.shape))
#         env = FireResetEnv(env)
#         print("FireResetEnv          : {}".format(env.observation_space.shape))
#         # env = MaxAndSkipEnv(env)
#         # print("MaxAndSkipEnv        : {}".format(env.observation_space.shape))
#         env = ResizeObservation(env, (84, 84))
#         print("ResizeObservation    : {}".format(env.observation_space.shape))
#         env = AddChannelDimension(env)  # Add channel dimension here
#         print("AddChannelDimension  : {}".format(env.observation_space.shape))
#         # env = GrayscaleObservation(env, keep_dim=True)
#         # print("GrayscaleObservation : {}".format(env.observation_space.shape))
#         env = ScaledFloatFrame(env)
#         print("ScaledFloatFrame     : {}".format(env.observation_space.shape))
#         # env = Monitor(env, allow_early_resets=False) # from stable baselines
#         # print("Monitor               : {}".format(env.observation_space.shape))

#         return env
#     return _init


# Create an evaluation environment (similar to our training env)
eval_env = make_vec_env(env_id=make_env(config["env_name"], allowed_actions= allowed_actions), n_envs=1)

# stack 4 frames
eval_env = VecFrameStack(eval_env, n_stack=4)
print("eval_env Post VecFrameStack Shape: {}".format(eval_env.observation_space.shape))

# convert back to PyTorch format (channel-first)
# eval_env = VecTransposeImage(eval_env)
eval_env = MyVecTransposeImage(eval_env)
print("eval_env MyVecTransposeImage Shape: {}".format(eval_env.observation_space.shape))
print("eval_env Final Observation Space: {}".format(eval_env.observation_space.shape))

In [ ]:
# obs = eval_env.reset()
# print(obs.shape)
# print(obs)
# print("Initial Obs Stats -> min:", obs.min(), "max:", obs.max(), "mean:", obs.mean())

In [ ]:
# define
# batch_size=30048,
# policy_kwargs = dict(net_arch=[512, 256])
# policy_kwargs=dict(net_arch=[512, 256], normalize_images=False)
model = PPO(config["policy_type"], 
            env, 
            verbose=0, 
            tensorboard_log=f"runs/{run.id}",
            batch_size=256, 
            learning_rate=2.5e-4, #2.5e-4, 0.001
            gamma=0.99,
            n_steps=128,
            n_epochs=4,
            clip_range=0.1,
            vf_coef=0.5,
            ent_coef=0.05, # 0.01
            policy_kwargs=dict(net_arch=[512, 256], normalize_images=False),
            device="cuda")


In [ ]:
# Create the evaluation callback
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=config["export_path"],  # directory to save the best model
    log_path=config["export_path"],              # evaluation logs
    eval_freq=500,                            # evaluate the model every 500,000 steps
    deterministic=True,
    render=False
)

In [ ]:
class GradientInspectionCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(GradientInspectionCallback, self).__init__(verbose)
    
    def _on_step(self) -> bool:
        # Access the policy network
        policy_net = self.model.policy
        
        # Iterate over the parameters to inspect gradients
        for name, param in policy_net.named_parameters():
            if param.grad is not None:
                grad_norm = param.grad.norm().item()
                self.logger.record(f"gradients/{name}_norm", grad_norm)
        
        return True  # Continue training


In [ ]:
class DebugObservationCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(DebugObservationCallback, self).__init__(verbose)

    def _on_step(self) -> bool:
        # Get the current observation
        observation = self.locals["new_obs"]  # Observations from the environment
        print("Observation mean:", np.mean(observation))
        print("Observation std:", np.std(observation))
        print("Observation min:", np.min(observation))
        print("Observation max:", np.max(observation))

        # Stop training if all observations are zero (optional)
        # if np.all(observation == 0):
        #     print("All observation values are zero. Exiting...")
        #     return False

        return True  # Continue training

In [ ]:
# # Initialize our custom callback
# wandb_best_model_callback = WandbBestModelCallback(
#     best_model_path=config["export_path"],
#     artifact_name="best_model",
#     project_name="donkey_kong_additional_reward",  # replace with your project name
#     entity=None,  # add entity if you have one
#     verbose=1
# )

In [ ]:
# Combine both callbacks
# callbacks = CallbackList([eval_callback, WandbCallback(verbose=2), GradientInspectionCallback(), DebugObservationCallback()])
callbacks = CallbackList([eval_callback, WandbCallback(verbose=2), GradientInspectionCallback()])

In [ ]:
# train
t0 = datetime.now()
model.learn(total_timesteps=config["total_timesteps"], callback=callbacks)
t1 = datetime.now()
print('>>> Training time (hh:mm:ss.ms): {}'.format(t1-t0))

# save and export model
model.save(config["export_path"] + config["model_name"])

Training time without parallezing env: >>> Training time (hh:mm:ss.ms): 1:01:58.577332

In [ ]:
# finish wandb project
wandb.finish()

In [ ]:
print("Finish")

In [ ]:
# import shutil
# shutil.rmtree("/kaggle/working/exports")
# os.remove("/kaggle/working/Breakout-v5.zip")

In [ ]:
# !zip -r folder_name.zip exports
# from google.colab import files
# files.download('folder_name.zip')

In [ ]:
# env = gym.make(env_name, obs_type="grayscale", render_mode=render_mode)
#         print("Standard Env.        : {}".format(env.observation_space.shape))
#         env = FireResetEnv(env)
#         print("FireResetEnv          : {}".format(env.observation_space.shape))
#         # env = MaxAndSkipEnv(env)
#         # print("MaxAndSkipEnv        : {}".format(env.observation_space.shape))
#         env = ResizeObservation(env, (84, 84))
#         print("ResizeObservation    : {}".format(env.observation_space.shape))
#         env = AddChannelDimension(env)  # Add channel dimension here
#         print("AddChannelDimension  : {}".format(env.observation_space.shape))
#         # env = GrayscaleObservation(env, keep_dim=True)
#         # print("GrayscaleObservation : {}".format(env.observation_space.shape))
#         env = ScaledFloatFrame(env)
#         print("ScaledFloatFrame     : {}".format(env.observation_space.shape))
#         env = Monitor(env, allow_early_resets=False) # from stable baselines
#         print("Monitor 